# Simple example working with video data

In [1]:
import findspark
findspark.init()

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import udf, size, col
from pyspark.sql.types import (
    ArrayType,
    FloatType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
)

spark = (
    SparkSession
    .builder
    .appName('rikai-quickstart')
    .config('spark.jars.packages', 'ai.eto:rikai_2.12:0.0.3')
    .master('local[*]')
    .getOrCreate()
)

## We'll work with a small set of music videos from The Hu

In [2]:
# Youtube video ids
the_hu = {
    'Sad But True': 'QpxA_ZxGX_M',
    'Sugaan Essena': 'YwutOqv4cGo',
    'The Great Chingghis Khaan': 'pD1gDSao1eA',
    'The Gereg': 'pu5jxc2pZtE',
    'Wolf Totem': 'jM8dCGIm6yc',
    'Yuve Yuve Yu': 'v4xZUr0BEfE'
}

In [3]:
from rikai.spark.types import YouTubeVideoType
from rikai.types import YouTubeVideo
import pandas as pd

pdf = (pd.DataFrame({'vid': the_hu})
      .reset_index()
      .rename(columns={'index':'name'}))

pdf['youtube_video'] = pdf.vid.apply(YouTubeVideo)

schema = StructType(
    [
        StructField("name", StringType(), False),
        StructField("vid", StringType(), False),        
        StructField("youtube_video", YouTubeVideoType(), False)
    ]
)

df = spark.createDataFrame(pdf, schema=schema)

### We can read/write Rikai format with native video types

In [4]:
df.coalesce(1).write.mode('overwrite').format('rikai').save('/tmp/thehu')

In [5]:
df = spark.read.format("rikai").load("/tmp/thehu/")
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- vid: string (nullable = true)
 |-- youtube_video: youTubeVideo (nullable = true)

+--------------------+-----------+--------------------+
|                name|        vid|       youtube_video|
+--------------------+-----------+--------------------+
|        Sad But True|QpxA_ZxGX_M|YouTubeVideo(vid=...|
|       Sugaan Essena|YwutOqv4cGo|YouTubeVideo(vid=...|
|           The Gereg|pu5jxc2pZtE|YouTubeVideo(vid=...|
|The Great Chinggh...|pD1gDSao1eA|YouTubeVideo(vid=...|
|          Wolf Totem|jM8dCGIm6yc|YouTubeVideo(vid=...|
|        Yuve Yuve Yu|v4xZUr0BEfE|YouTubeVideo(vid=...|
+--------------------+-----------+--------------------+



In [6]:
yt = df.first()['youtube_video']
yt

YouTubeVideo(QpxA_ZxGX_M)

In [7]:
v = yt.get_stream()
v

VideoStream(uri=https://r2---sn-n4v7knls.googlevideo.com/videoplayback?expire=1611828202&ei=ijcSYILiM8bCkga6gbcI&ip=69.181.105.47&id=o-APePtLrJwGQJJQZZ68mDHt7DiV1itojG2y4Ear2W4TxA&itag=18&source=youtube&requiressl=yes&mh=T9&mm=31%2C26&mn=sn-n4v7knls%2Csn-a5meknek&ms=au%2Conr&mv=m&mvi=2&pl=22&initcwndbps=2003750&vprv=1&mime=video%2Fmp4&ns=GYhKdLCvva5Jh-u93IcAEPMF&gir=yes&clen=26001396&ratebypass=yes&dur=374.398&lmt=1610809261405477&mt=1611806447&fvip=2&c=WEB&txp=5530434&n=-DBit-uBzZnsTP5_mM6N&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cns%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRQIgdeR0GQm1nBN0fYhFteUXt72sXH3gyKBQDMDZqVv4aaICIQC9jxjWDYKJ3wGJI9_BV9Djp88eNLAvK0JQ4ZnIv3ze6g%3D%3D&sig=AOq0QJ8wRAIgTonKS_GQEyNZ9qbHdWtiqkQcIUpKOP_3UWI2xwMEBLQCIGChZVxlz-lhMAeIA7NjJ693Qaz36GY4Wzoe6ye4_6_A)

## We make it easy to sample the frames for deep learning

In [8]:
from rikai.torch import DataLoader

data_loader = DataLoader(
    '/tmp/thehu',
    batch_size=1,
    shuffle=True
)
max_frames = 1 #
for batch in data_loader:
    yt = batch[0]['youtube_video']
    print(yt)
    for fno, frame in enumerate(yt.get_stream()):
        if fno >= max_frames:
            break
        print('fno: ', fno)
        print('frame: ', frame)

2021-01-27 20:03:23,510 INFO Rikai (dataset.py:108): Loading parquet files: ['file:///tmp/thehu/part-00000-26b28f78-0cfb-43ae-be9d-0c97fe72878c-c000.snappy.parquet']


YouTubeVideo(v4xZUr0BEfE)
fno:  0
frame:  [[[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
YouTubeVideo(jM8dCGIm6yc)
fno:  0
frame:  [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
YouTubeVideo(pD1gDSao1eA)
fno:  0
frame:  [[[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [0 0 0]
